# Day 6 - Date and Time Challenges
Working with date and time data is always a struggle, because there are many different notations and formats as well as the timezone issue. So today I want to check, how the `pysparl.sql.functions` module can help me here.

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession\
   .builder\
   .getOrCreate()

There are just two kind of objects regarding date and time known to Spark
1. `DateType`: which refers to a calender
1. `TimeStampType`: which is a date extended by a time releated to a timezone

By default, Spark derives the current date and timestamp from the local machine settings, unless it get's overruled by `SparkConf` settings.

In [37]:
import pyspark.sql.functions

df = spark\
    .range(1)\
    .withColumn("Today", current_date())\
    .withColumn("Now", current_timestamp())

df.show(truncate=False)

+---+----------+-----------------------+
|id |Today     |Now                    |
+---+----------+-----------------------+
|0  |2020-04-16|2020-04-16 12:47:08.977|
+---+----------+-----------------------+



## Decomposing Date and Timestamp Objects
If I need to check for a particular part of a date/timestap, maybe to get all data records created in Apr-2020, I can decompose these objects by one of the following functions.

In [70]:
df.select(
    "now",
    year("Now").alias("year"),
    quarter("Now").alias("quarter"),
    month("now").alias("month"),
    dayofmonth("now").alias("day of month"),
    hour("now").alias("hours"),
    minute("now").alias("minutes"),
    minute("now").alias("seconds")
).show(vertical=True, truncate=False)

-RECORD 0-------------------------------
 now          | 2020-04-16 13:54:04.238 
 year         | 2020                    
 quarter      | 2                       
 month        | 4                       
 day of month | 16                      
 hours        | 13                      
 minutes      | 54                      
 seconds      | 54                      



I can even derive some further attributes from a given date. Especially the `last_day()` function can help me implementing month-end reports.

In [112]:
df.select(
    "now",
    weekofyear("now").alias("week of year"),
    dayofweek("now").alias("day of week"),
    last_day("now").alias("month end"),
    next_day("today", "Sun").alias("next Sunday")
).show(vertical=True, truncate=False)

-RECORD 0-------------------------------
 now          | 2020-04-16 15:42:52.371 
 week of year | 16                      
 day of week  | 5                       
 month end    | 2020-04-30              
 next Sunday  | 2020-04-19              



## Date Shifts and Date Period Calculation

Spark provides native support for calulations of time periodes on daily or monthly basis.

In [113]:
df.select(
    "today",
    date_add("today", 2).alias("day after tomorrow"),
    date_add("today", -1).alias("yesterday"),
    date_sub("today", 1).alias("yesterday"),
    add_months("today", 1).alias("same day next month"),
    add_months("today", -1).alias("same day prev. month"),
    "now",
    date_add("now", 1).alias("tomorrow at same time"),
    date_sub("now", 1).alias("yesterday at same time"),
    add_months("now", 1).alias("next month"),
    datediff(date_add("today", 1), date_sub("today", 1)).alias("# of days between yesterday and tomorrow"),
    months_between("today", date_add("today", 77)).alias("# of months between two dates"),
    months_between("today", date_sub("today", 77)).alias("# of months between two dates")
).show(vertical=True, truncate=False)

-RECORD 0-----------------------------------------------------------
 today                                    | 2020-04-16              
 day after tomorrow                       | 2020-04-18              
 yesterday                                | 2020-04-15              
 yesterday                                | 2020-04-15              
 same day next month                      | 2020-05-16              
 same day prev. month                     | 2020-03-16              
 now                                      | 2020-04-16 15:43:02.217 
 tomorrow at same time                    | 2020-04-17              
 yesterday at same time                   | 2020-04-15              
 next month                               | 2020-05-16              
 # of days between yesterday and tomorrow | 2                       
 # of months between two dates            | -2.5483871              
 # of months between two dates            | 2.5483871               



It's quite counter-intuitive to me that the month diff is negative when the second date is later than the first date and vice versa. The calculation is simply date1 - date2 and since earlier dates are smaller than later ones,  the negative result obvious is absolutely intuitive for mathematicians.

I should keep also in mind, that when applying these funtions to timestamps, they get truncated to dates and timestamps by myself.

The `date_sub()` function is redundant because date_add() accepts also negative day shifts.

In [76]:
df.select(
    "today",
    date_trunc("yyyy", "today").alias("yyyy"),
    date_trunc("year", "today").alias("year"),
    date_trunc("year", "today").alias("yy"),
    date_trunc("quarter", "today").alias("quarter"),
    date_trunc("month", "today").alias("month"),
    date_trunc("mon", "today").alias("mon"),
    date_trunc("week", "today").alias("week"),
    date_trunc("mm", "today").alias("mm"),
    "now",
    date_trunc("yyyy", "now").alias("yyyy"),
    date_trunc("year", "now").alias("year"),
    date_trunc("year", "now").alias("yy"),
    date_trunc("quarter", "now").alias("quarter"),
    date_trunc("month", "now").alias("month"),
    date_trunc("mon", "now").alias("mon"),
    date_trunc("week", "now").alias("week"),
    date_trunc("mm", "now").alias("mm"),
    date_trunc("day", "now").alias("day"),
    date_trunc("dd", "now").alias("dd"),
    date_trunc("hour", "now").alias("hour"),
    date_trunc("minute", "now").alias("minute"),
    date_trunc("second", "now").alias("second"),
    date_trunc("quarter", "today").alias("quarter"),
).show(vertical=True, truncate=False)

-RECORD 0-------------------------
 today   | 2020-04-16             
 yyyy    | 2020-01-01 00:00:00    
 year    | 2020-01-01 00:00:00    
 yy      | 2020-01-01 00:00:00    
 quarter | 2020-04-01 00:00:00    
 month   | 2020-04-01 00:00:00    
 mon     | 2020-04-01 00:00:00    
 week    | 2020-04-13 00:00:00    
 mm      | 2020-04-01 00:00:00    
 now     | 2020-04-16 14:06:22.63 
 yyyy    | 2020-01-01 00:00:00    
 year    | 2020-01-01 00:00:00    
 yy      | 2020-01-01 00:00:00    
 quarter | 2020-04-01 00:00:00    
 month   | 2020-04-01 00:00:00    
 mon     | 2020-04-01 00:00:00    
 week    | 2020-04-13 00:00:00    
 mm      | 2020-04-01 00:00:00    
 day     | 2020-04-16 00:00:00    
 dd      | 2020-04-16 00:00:00    
 hour    | 2020-04-16 14:00:00    
 minute  | 2020-04-16 14:06:00    
 second  | 2020-04-16 14:06:22    
 quarter | 2020-04-01 00:00:00    



## Type Conversion: String vs. Date/Timstamps
Especially during data import or export of dates/timestamps, I often have to read from strings or write to strings. 

### Reading Strings:

In [80]:
stringDF = spark.range(1).withColumn("ts string", lit("2020-04-16 14:16:23"))
stringDF.show(truncate=False)

+---+-------------------+
|id |ts string          |
+---+-------------------+
|0  |2020-04-16 14:16:23|
+---+-------------------+



When I read from file with option("inferSchema", "true") Spark can identifiy most of the common date and datetime notations and derive the correspondig column type. If the notation is unknon to Spark, it will set the column to DataTypeString. In that case I can convert these strings to dates or timestamps during a subsequent transformation. This requires a string format according to the Java <a href="https://docs.oracle.com/javase/tutorial/i18n/format/simpleDateFormat.html">SimpleDateFormats</a>

**Note:** Internally Spark works with Java dates and timestamps.

In [88]:
stringDF.select(
        "ts string",
        to_timestamp("ts string", "yyyy-MM-dd HH:mm:ss").alias("converted to ts"),
        to_date("ts string", "yyyy-MM-dd HH:mm:ss").alias("converted to date")
).show(truncate=False)

+-------------------+-------------------+-----------------+
|ts string          |converted to ts    |converted to date|
+-------------------+-------------------+-----------------+
|2020-04-16 14:16:23|2020-04-16 14:16:23|2020-04-16       |
+-------------------+-------------------+-----------------+



### Writing as Strings
To convert either a date or a timestamp into a string, again I have to specify the format according to the Java <a href="https://docs.oracle.com/javase/tutorial/i18n/format/simpleDateFormat.html">SimpleDateFormats</a>

In [90]:
df.select(
    date_format("today", "dd.MM.yyyy").alias("date string"),
    date_format("now", "dd.MM:yyyy HH:mm.ss").alias("ts string")
).show(truncate=False)

+-----------+-------------------+
|date string|ts string          |
+-----------+-------------------+
|16.04.2020 |16.04:2020 14:53.11|
+-----------+-------------------+



## Timezone Conversion
Processing timestamp gets even more complicated, when I need to load data from source systems, which are located in different timezones. When I want to analyse flight durations, I should consider that the timezone offsets between daparture time and arrival time. Otherwise in my dataresult a flight from Hamburg to London might take just 5 minutes.

So to ensure proper time period calculations I should normalize all timestamps to a common timezone which is the *Unified Time Coordinated (UTC)*.

In [101]:
utcDF = df.select(
    "now",
    to_utc_timestamp("now", "CET").alias("CET ts in UTC"),
    to_utc_timestamp("now", "CET").alias("CEST ts ts in UTC"),
    to_utc_timestamp("now", "Europe/Berlin").alias("Europe/Berlin ts in UTC"),
)

utcDF.show(truncate=False)

+-----------------------+-----------------------+-----------------------+-----------------------+
|now                    |CET ts in UTC          |CEST ts ts in UTC      |Europe/Berlin ts in UTC|
+-----------------------+-----------------------+-----------------------+-----------------------+
|2020-04-16 15:25:19.675|2020-04-16 13:25:19.675|2020-04-16 13:25:19.675|2020-04-16 13:25:19.675|
+-----------------------+-----------------------+-----------------------+-----------------------+



Obviously, Spark is smart enough to identify, that the given timestamp is in the day-light-saving period where CET = UTC+1 get's shifted to CEST = UTC+2, so in all cases Spark applies the 'Europe/Berlin' rules regardless whether I define this rule rexplicitly or I pass the CET or CEST timezone.

If I need to localize UTC timestamps, I just need to apply the reverse function.



In [102]:
utcDF.select(
    from_utc_timestamp("Europe/Berlin ts in UTC", "Europe/Berlin").alias("local ts")
).show(truncate=False)

+-----------------------+
|local ts               |
+-----------------------+
|2020-04-16 15:25:32.204|
+-----------------------+



The Unix timestamp is another alternative for timestamp harmonization:

**from_unixtime(timestamp, format='yyyy-MM-dd HH:mm:ss')**
Converts the number of seconds from unix epoch (1970-01-01 00:00:00 UTC) to a string representing the timestamp of that moment in the current system time zone in the given format.


**unix_timestamp(timestamp=None, format='yyyy-MM-dd HH:mm:ss')**
Convert time string with given pattern (‘yyyy-MM-dd HH:mm:ss’, by default) to Unix time stamp (in seconds), using the default timezone and the default locale, return null if fail.